# Jupyter Notebook

## Assignment : In Lesson : Utility software 2

Open "Utility Software Lesson 2.ipynb" and give it permission

Press "Shift + Enter" to run each block (updating thing you've edited or changed)


# Utility Software : Compression

Utility Software that compresses is almost always **Lossless** as you need to get the files back exactly like there orginal.

## When is lossy compression useful?

Double click here to edit

Press "Shift + Enter" to process a box and move on to the next bit. 

## Set up

Use "Shift + Enter" to run the next 2 blocks, this will get everything we need set up.


In [ ]:
# Install 2 python libraries
import micropip
await micropip.install(["python-rle", "pycountry"]) # Install required libraries

In [ ]:
# Import libraries
import rle
from js import fetch
import json
from PIL import Image, ImageDraw
import numpy as np
import pycountry
import sys
from IPython.display import Image as ImageDisplay

class NpEncoder(json.JSONEncoder):
    """Work around for numpy to JSON"""
    def default(self, obj):
        if isinstance(obj, np.bool_):
            return bool(obj)
        if isinstance(obj, (np.floating, np.complexfloating)):
            return float(obj)
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.string_):
            return str(obj)
        return super(NpEncoder, self).default(obj)
         
async def load_file_into_in_mem_filesystem(url, fn=None):
    """Load a file from a URL into an in-memory filesystem."""
    # Create a filename if required
    fn = fn if fn is not None else url.split("/")[-1]
    # Fetch file from URL
    res = await fetch(url)
    # Buffer it
    buffer = await res.arrayBuffer()
    # Write file to in-memory file system
    open(fn, "wb").write(bytes(buffer.valueOf().to_py()))
 
    return fn


async def fetch_flag(text):
    """Get a flag from the web by eith 2 letter code or searching for that code using the text """
    if (len(text) > 2):
        code = pycountry.countries.search_fuzzy(text)[0].alpha_2
    code = code.lower()
    url = "https://raw.githubusercontent.com/hampusborgos/country-flags/main/png100px/"+code+".png"
    print(url)
    flag_file = await load_file_into_in_mem_filesystem(url)
    return flag_file


def compress_VertRleOfHoriRle(dat):
    """Compress dat by doing a run length encoding up the column after they been RLEd along the rows already
    Return string"""
    lines = []
    for x in dat:
        x_rle = rle.encode(x.tolist())
        lines.append(x_rle)
    out = rle.encode(lines)
    return json.dumps(out, cls=NpEncoder, separators=(',', ':'))

def decompress_VertRleOfHoriRle(text):
    """Decode the RLE Vertically first, the horizontally creating a grid of pixels"""
    comp = json.loads(text)
    out = [rle.decode(x[0], x[1]) for x in rle.decode(comp[0], comp[1])]
    return out

def compress_HoriRleOfVertRle(dat):
    """Compress dat by doing a run length encoding along the row after they been RLEd up the columns already
    Return String"""
    cols = []
    for col in dat.T:
        cols.append(rle.encode(col.tolist()))
    out = rle.encode(cols)
    return json.dumps(out, cls=NpEncoder, separators=(',', ':'))

def decompress_HoriRleOfVertRle(text):
    """Decode the RLE Horizontally first, placing it in a grid"""
    comp = json.loads(text)
    dat = []
    cols = rle.decode(comp[0], comp[1])
    for x in range(len(cols)):
        col = rle.decode(cols[x][0], cols[x][1])
        for y in range(len(col)):
            if x == 0:
                dat.append([])
            dat[y].append(col[y])
    return  dat

def createImage(data, palette):
    image = Image.new("P", (len(data[0]), len(data)))
    image.putpalette(palette)
    for x in range(image.width):
        for y in range(image.height):
            image.putpixel((x,y), data[y][x])
    return image

# rle-encode.py

def rle_encode(data):
    encoding = ''
    prev_char = ''
    count = 1

    if not data: return ''

    for char in data:
        # If the prev and current characters
        # don't match...
        if char != prev_char:
            # ...then add the count and character
            # to our encoding
            if prev_char:
                encoding += str(count) + prev_char
            count = 1
            prev_char = char
        else:
            # Or increment our counter
            # if the characters do match
            count += 1
    else:
        # Finish off the encoding
        encoding += str(count) + prev_char
        return encoding

# rle-decode.py

def rle_decode(data):
    decode = ''
    count = ''
    for char in data:
        # If the character is numerical...
        if char.isdigit():
            # ...append it to our count
            count += char
        else:
            # Otherwise we've seen a non-numerical
            # character and need to expand it for
            # the decoding
            decode += char * int(count)
            count = ''
    return decode

def rle_results(plain_text): 
    encoded = rle_encode(plain_text)
    decoded = rle_decode(encoded)
    print("Original =", plain_text, "Orginal Length", len(plain_text), "bytes (assuming ASCII)")
    print("Encoded  =", encoded, "Encoded Length", len(encoded), "bytes (assuming ASCII)")
    print("Decoded  =", decoded)

async def flags_rle_results(country_name, debug = False):

    file_name = await fetch_flag(country_name)
    print("country_name= ", country_name, "file_name =", file_name)

    im = Image.open(file_name)
    im2 = im.quantize(dither = Image.Dither.NONE)
    
    #display(im2) # Show the image
    palette = im2.getpalette() 
    dat = np.asarray(im2)
    print("Size of original image approx", (im2.width * im2.height) + len(palette), "bytes") 

    HVtext = compress_HoriRleOfVertRle(dat)
    print("HoriRleOfVertRle", "length =", len(HVtext), "bytes")
    if debug:
        print( "HVtext =", HVtext)
    uncompressed = decompress_HoriRleOfVertRle(HVtext)
    #im3 = createImage(decompress_HoriRleOfVertRle(HVtext), im2.getpalette())
    #display(im3)

    VHtext = compress_VertRleOfHoriRle(dat)
    print("HoriRleOfVertRle", "length =", len(VHtext), "bytes")
    if debug:
        print( "VHtext =", VHtext)
    #im4 = createImage(decompress_VertRleOfHoriRle(VHtext), im2.getpalette())
    return im2, file_name
    

## Simple Compression : Run Length Encoding 

On paper 
Steps :
1. Get Text
2. Start on left
3. Get next letter
4. How many of this letter next are next to each other (the run)?
5. Write down the Run Length and the Letter
6. Go To 3

```plain_text = "ABBCCCDDDDEEEEEFFFFFFGGGGGGG"```

### Task : 

1. Follow the instructions 
2. Double click the code below and fill in ```hand_encoded="1A"``` The first run is done for you.
3. Once you have it correct "Shift + Enter" will run it

In [ ]:
plain_text = "ABBCCCDDDDEEEEEFFFFFFGGGGGGG"

# TODO : Complete this Run Length encoded string by hand
hand_encoded = "1A"

encoded = rle_encode(plain_text)
assert hand_encoded == encoded, "That's not quite right" 
encoded

#### Evidence

In [ ]:
rle_decode(encoded)

#### Is it any good?

In [ ]:
plain_text = "ABBCCCDDDEEEEFFFFFGGGGGG"
rle_results(plain_text)


In [ ]:
plain_text = "Our Deepest Fear Is Not That We Are Inadequate. Our Deepest Fear Is That We Are Powerful Beyond Measure"
rle_results(plain_text)

### Is Run Length Encoding Good for Text?

Double Click and answer here.

## Loading an Image

In [ ]:
country_name = 'Germany'

file_name = await fetch_flag(country_name)
print("file_name =", file_name)

im = Image.open(file_name)
im2 = im.quantize(dither = Image.Dither.NONE)

await ImageDisplay(im2, filename=file_name) 


## Looking at the data



### Paint by Numbers

In [ ]:
palette = im2.getpalette() 
print("Palette", palette[:60])
for colour in range(3):
    print("Colour", colour, "Red =",palette[(colour*3)], "Green =",palette[(colour*3)+1], "Blue =",palette[(colour*3)+2])

dat = np.asarray(im2)

print("The colours in the top row", dat[0]) # Show the data of the bottom row

In [ ]:
#np.set_printoptions(threshold=sys.maxsize) # Print every bit
#dat

## Compressing an Image : RLE H of V

Run Length encoding each Column upwards and then RLE those columns again sidewards 

In [ ]:
print("Size of original image approx", (im2.width * im2.height) + len(palette), "bytes") 

HVtext = compress_HoriRleOfVertRle(dat)
print("HoriRleOfVertRle", "length =", len(HVtext), "bytes")
print( "HVtext =", HVtext)
uncompressed = decompress_HoriRleOfVertRle(HVtext)
im3 = createImage(decompress_HoriRleOfVertRle(HVtext), im2.getpalette())

ImageDisplay(im3, filename=file_name) 


- This is compressing to a string representation of Arrays (JSON) (we could use binary to make it even small).
- In the case of Germany  
    - HVtext = [[[[2,1,0],[20,20,20]]],[**100**]] a 100 columns of ..
        - HVtext = [[[[**2**,1,0],[**20**,20,20]]],[100]] 20 pixels of palette colour 2 [0,0,0] Black
        - HVtext = [[[[2,**1**,0],[20,**20**,20]]],[100]] 20 pixels of palette colour 1 [0,221,0] Red
        - HVtext = [[[[2,1,**0**],[20,20,**20**]]],[100]] 20 pixels of palette colour 1 [255, 206, 0] Yellow


## Compressing an Image : RLE V of H

Run Length encoding each row left to right then RLE those together upwards 

In [ ]:
VHtext = compress_VertRleOfHoriRle(dat)
print("HoriRleOfVertRle", "length =", len(VHtext), "bytes")
print( "VHtext =", VHtext)
im4 = createImage(decompress_VertRleOfHoriRle(VHtext), im2.getpalette())

ImageDisplay(im3, filename=file_name) 


- In the case of Germany  
    - HVtext = [[**[[2],[100]]**,[[1],[100]],[[0],[100]]],[**20**,20,20]] 20 rows of 100 pixels of palette colour 2 [0,0,0] Black
    - HVtext = [[[[2],[100]],**[[1],[100]]**,[[0],[100]]],[20,**20**,20]] 20 rows of 100 pixels of palette colour 1 [0,221,0] Red
    - HVtext = [[[[2],[100]],[[1],[100]],**[[0],[100]]**],[20,20,**20**]] 20 rows of 100 pixels of palette colour 0 [255, 206, 0] Yellow

### Remix Germany


Change the flag around maybe change the size (it might crash don't worry.)

In [ ]:
palette = [255, 206, 0, 221, 0, 0, 0, 0, 255]
remix = "[[[[2,1,0],[20,40,20]]],[100]]"
im_remix = createImage(decompress_HoriRleOfVertRle(remix), palette)
file_name = "im_remix.png"
im_remix.save(file_name)
ImageDisplay(im_remix, filename=file_name)

### Is Run Length Encoding Good for Images ?

In [ ]:
im, file_name = await flags_rle_results(country_name = 'France')
ImageDisplay(im, filename=file_name)

In [ ]:
im, file_name = await flags_rle_results(country_name = 'Mexico')
ImageDisplay(im, filename=file_name)

In [ ]:
im, file_name = await flags_rle_results(country_name = 'Iran')
ImageDisplay(im, filename=file_name)

In [ ]:
im, file_name = await flags_rle_results(country_name = 'Brazil')
ImageDisplay(im, filename=file_name)

In [ ]:
im, file_name = await flags_rle_results(country_name = 'Britain')
ImageDisplay(im, filename=file_name)

In [ ]:
im, file_name = await flags_rle_results(country_name = 'Afghanistan')
ImageDisplay(im, filename=file_name)

In [ ]:
im, file_name = await flags_rle_results(country_name = 'Pitcairn')
ImageDisplay(im, filename=file_name)

### Is Run Length Encoding Good for Images ?

Double Click and answer here.


#### Explain any exceptions to that.

Double Click and answer here.


## Utility Software : Compression

- These examples are all in code working on small bits of data.
- There are loads of other types of compression suitable to other type of data (Text etc.)
- Utility Software will compress and decompress whole files / folders 


## Utility Software : Compression : Examples

List as many Compression Tools as you can (Double click to edit)

# Encryption Software

Set up

In [ ]:
import micropip
await micropip.install(["cryptography", "rsa"])

## Utility Software : Encryption

Encryption is the process of encoding the data. i.e converting plain text into ciphertext. This conversion is done with a key called an encryption key.

## Decryption:

Decryption is the process of decoding the encoded data. Converting the ciphertext into plain text. This process requires a key that we used for encryption.

### Keys
We require a key for encryption. There are two main types of keys used for encryption and decryption. They are Symmetric-key and Asymmetric-key.

## What is Symmetry ?


### Symmetric-key Encryption

Use the same key to encrypt and then decrypt data.

In [ ]:
from cryptography.fernet import Fernet

# we will be encrypting the below string.
message = "10X/Co1 is learning symmetric encryption"

# generate a key for encryption and decryption. You can use fernet to generate the key or use random key generator

key = Fernet.generate_key()

print("key =", key)

# Instance the Fernet class with the key

fernet = Fernet(key)

# then use the Fernet class instance to encrypt the string string must be encoded to byte string before encryption
encMessage = fernet.encrypt(message.encode())

print("original string =", message)
print("encrypted string =", encMessage)

### Then Decrypt it

In [ ]:
# decrypt the encrypted string with the Fernet instance of the key, that was used for encrypting the string
# encoded byte string is returned by decrypt method, so decode it to string with decode methods
decMessage = fernet.decrypt(encMessage).decode()

print("decrypted string = ", decMessage)


### Asymmetric-key Encryption

Using different keys to encrypt and decrypt data.

In [ ]:
import rsa

# generate public and private keys with rsa.newkeys method,this method accepts 
# key length as its parameter key length should be at least 16
publicKey, privateKey = rsa.newkeys(512)

# this is the string that we will be encrypting
message = "10X/Co1 is learning asymmetric encryption"

# rsa.encrypt method is used to encrypt string with public key string should be 
# encode to byte string before encryption with encode method
encMessage = rsa.encrypt(message.encode(), publicKey)

print("Public Key = ", publicKey)
print("Original String =", message)
print("Encrypted String =", encMessage)


### Asymmetric-key Decryption

In [ ]:
# the encrypted message can be decrypted with ras.decrypt method and private key
# decrypt method returns encoded byte string, use decode method to convert it to string
# public key cannot be used for decryption
decMessage = rsa.decrypt(encMessage, privateKey).decode()

print("Decrypted String =", decMessage)



## Save and turn in